In [436]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)

def split_str(str):
    word = ""
    for char in str:
        if not char.isdigit():
            word += char
    return word

def remove_symb(str):
    word = ""
    symbol = "("
    for char in str:
        if not char == symbol:
            word += char
        else:
            break
    word = word.strip()
    return word

def answer_one():
    file = "Energy Indicators.xls"
    energy = pd.read_excel(file,skiprows= 17, nrows= 227 )
    del energy['Unnamed: 0']
    del energy['Unnamed: 1']
    energy.rename( columns= {'Unnamed: 2':'Country',
                         "Petajoules": 'Energy Supply',
                         "Gigajoules": 'Energy Supply per Capita', "%":'% Renewable'}, inplace = True) 
    energy["Country"] = energy["Country"].apply(split_str)
    energy["Country"] = energy["Country"].apply(remove_symb)
    energy["Energy Supply"] = energy["Energy Supply"] * 1000000
    energy["Energy Supply"] = energy["Energy Supply"].apply(lambda x: x if type(x) is int else None)
    energy["Country"] = energy["Country"].apply(lambda x: "South Korea" if x == "Republic of Korea" else x)
    energy["Country"] = energy["Country"].apply(lambda x: "United States" if x == "United States of America" else x)
    energy["Country"] = energy["Country"].apply(lambda x: "United Kingdom" if x == "United Kingdom of Great Britain and Northern Ireland" else x)
    energy["Country"] = energy["Country"].apply(lambda x: "Hong Kong" if x == "China, Hong Kong Special Administrative Region" else x)
    
    file_gdp = "world_bank.csv"
    GDP = pd.read_csv(file_gdp, sep = ',',
                     dtype = None, skiprows=4, encoding='utf-8')

    GDP.rename( columns= {'Country Name':'Country'}, inplace = True) 

    GDP["Country"] = GDP["Country"].apply(lambda x: "South Korea" if x == "Korea, Rep." else x)
    GDP["Country"] = GDP["Country"].apply(lambda x: "Iran" if x == "Iran, Islamic Rep." else x)
    GDP["Country"] = GDP["Country"].apply(lambda x: "Hong Kong" if x == "Hong Kong SAR, China" else x)
    gdp = GDP[["Country", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015"]]
    
    ScimEn_file = "country_rank.xlsx"
    ScimEn = pd.read_excel(ScimEn_file)
    del ScimEn["Region"]
    ScimEn_rows = ScimEn.iloc[0:15]
    merge_1 = ScimEn_rows.merge(energy, how="inner", on = 'Country')
    result = merge_1.merge(gdp, how="left", on = 'Country')
    result = result.set_index(['Country'])
    return result

answer_one()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,303064,301778,3036531,2092737,10.020,273,127191000000.000,93,19.755,2752131773358.920,3550342737009.530,4594307032667.980,5101703073088.970,6087163874510.730,7551500124197.170,8532229986993.650,9570406235659.641,10475682920597.699,11061553079871.500
United States,2,184851,181106,2623922,819242,14.190,389,90838000000.000,286,11.571,13815586948000.000,14474226905000.000,14769857911000.000,14478064934000.000,15048964444000.000,15599728123000.000,16253972230000.000,16843190993000.000,17550680174000.000,18206020741000.000
India,3,60257,58589,590570,213760,9.800,198,33195000000.000,26,14.969,940259888792.141,1216735441524.860,1198895582137.510,1341886602798.690,1675615335600.560,1823049927771.460,1827637859135.700,1856722121394.530,2039127446298.550,2103587813812.750
Japan,4,52780,52281,557023,132113,10.550,203,18984000000.000,149,10.233,4601663122649.920,4579750920354.810,5106679115127.300,5289493117993.890,5759071769013.110,6233147172341.350,6272362996105.030,5212328181166.180,4896994405353.290,4444930651964.180
United Kingdom,5,47141,45928,748994,132737,15.890,244,7920000000.000,124,10.600,2717059797608.100,3106182472989.200,2938882352941.180,2425798410967.440,2491110093454.180,2674891473843.810,2719158341005.790,2803291406029.930,3087165602962.860,2956573778737.760
Germany,6,42343,41464,528645,115432,12.480,211,13261000000.000,165,17.902,2994703642023.530,3425578382921.580,3745264093617.190,3411261212652.340,3399667820000.010,3749314991050.590,3527143188785.160,3733804649549.030,3889093051023.520,3357585719351.560
Russian Federation,7,39424,39189,142937,67935,3.630,96,30709000000.000,214,17.289,989930542278.695,1299705764823.620,1660846387624.780,1222644282201.860,1524917468442.010,2045925608274.370,2208295773643.150,2292473246621.080,2059241965490.830,1363481063446.770
Canada,8,35588,34940,665415,113363,18.700,246,10431000000.000,296,61.945,1319264809590.970,1468820407783.260,1552989690721.650,1374625142157.290,1617343367486.260,1793326630174.520,1828366481521.600,1846597421834.980,1805749878439.940,1556508816217.140
Italy,9,31260,29959,433388,105641,13.860,177,6530000000.000,109,33.667,1949551719389.640,2213102482751.460,2408655348718.590,2199928804118.630,2136099955236.670,2294994296589.500,2086957656821.600,2141924094298.560,2162009615996.540,1836637711060.550


In [437]:
def answer_two():
    Top15 = answer_one()
    years = [str(year) for year in range(2006, 2016)]
    avgGDP =  Top15[years].apply(lambda x: sum(x)/len(x), axis=1)
    avgGDP.sort_values(ascending=False, inplace=True)
    return avgGDP
answer_two()

Country
United States        15704029340300.000
China                 6927702083795.578
Japan                 5239642145206.906
Germany               3523341675097.451
United Kingdom        2792011373054.025
France                2691336579143.638
Italy                 2142986168498.174
Brazil                1988889078184.339
Russian Federation    1666746210284.716
Canada                1616359264592.761
India                 1602351801926.675
Spain                 1400726627860.776
South Korea           1221371619241.444
Australia             1207106159961.251
Iran                   456326122712.661
dtype: float64

In [438]:
def answer_three():
    Top15 = answer_one()
    avg = answer_two().index[5]
    diff = Top15.loc[avg]["2015"] - Top15.loc[avg]["2006"]
    return diff

answer_three()

118652421857.7998

In [450]:
def answer_four():
    Top15 = answer_one()
    ratio = Top15[["Citations", "Self-citations"]].apply(lambda x: x[1]/x[0], axis=1)
    max_ = ratio.max()
    ind = ratio[ratio == max_].index[0]
    Top15["Citations ratio"] = ratio
    
    return ind, max_

answer_four()

('China', 0.689186772669207)

In [453]:
def answer_five():
    Top15 = answer_one()
    capita = Top15[["Energy Supply", "Energy Supply per Capita"]].apply(lambda x: x[0]/x[1], axis=1)
    Top15["Population"] = capita
    capita.sort_values(ascending=False, inplace=True)
    index = capita[capita == capita[2]].index[0]
    
    return index
answer_five()

'United States'

In [454]:
def answer_six():  # don't understand the meaning of colums and what we have to find out
    Top15 = answer_one()
    capita = Top15[["Energy Supply", "Energy Supply per Capita"]].apply(lambda x: x[0]/x[1], axis=1)
    Top15["Population"] = capita
    documents_per_person = Top15[["Citable documents", "Population"]].apply(lambda x: x[0]/x[1], axis=1)
    Top15["Documents per person"] = documents_per_person

    return

answer_six()

In [468]:
def answer_seven():
    Top15 = answer_one()
    capita = Top15[["Energy Supply", "Energy Supply per Capita"]].apply(lambda x: x[0]/x[1], axis=1)
    Top15["Population"] = capita
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    continents = pd.DataFrame(ContinentDict.items(), columns = ["Country", "Continent"])
    appended = continents.merge(Top15, how="inner", on = 'Country')
    
    table = appended.groupby(['Continent'])['Population'].agg(['size', 'sum', "mean", "std"])
    
    return table

answer_seven()

,size,sum,mean,std
Continent,,,,
Asia,5,2898666386.611,579733277.322,679097888.366
Australia,1,23316017.316,23316017.316,NaN
Europe,6,457929667.216,76321611.203,34647667.066
North America,2,352855249.480,176427624.740,199669644.857
South America,1,205915254.237,205915254.237,NaN
